In [1]:
!pip install bs4
!pip install sqlalchemy
!pip install requests
!pip install pandas
!pip install firecrawl 


  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [bs4]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 372.6 kB/s eta 0:00:00a 0:00:01
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]2/5 [charset-normalizer]
  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 547.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
  Using cached firecrawl-2.5.4-py3-none-any.whl.metadata (7.2 kB)
  Using cached websockets-15.0.1-cp312-cp312-macosx_11_0

In [ ]:
from bs4 import BeautifulSoup
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key='fc-e6d03fc937144bc2a873053cfecd5987')
target_url = 'https://www.indeed.com/jobs?q=financial+analyst&l=&from=searchOnHP&vjk=6404f31c2a94102c'

In [ ]:
# Scrape the raw HTML
scrape_result = app.scrape_url(target_url, formats=['html'])


# Access the HTML content from the response object
html_content = scrape_result.html  # ✅ This is the correct attribute

# Parse with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the first job listing
first_job = soup.select_one('a[data-jk]')
company_location = soup.select_one('div.company_location')


# Extract data
if first_job:
    job_title = first_job.get_text(strip=True)
    job_href = first_job.get('href')
    full_link = f"https://www.indeed.com{job_href}"
    company_location_text = company_location.get_text(strip=True) if company_location else "N/A"
    # Print the extracted data
    print("Company Location:", company_location_text)

    print("First Job Title:", job_title)
    print("Link:", full_link)
else:
    print("No job found.")

Company Location: Arrivia, Inc.Scottsdale, AZ
First Job Title: Financial Analyst
Link: https://www.indeed.comhttps://www.indeed.com/rc/clk?jk=ddfb81ba0e36015e&bb=aP_xwJufGSlv-vh_kM2-3oK67dsFSTGHQrE0sHmtY0-Cu3pefT-A0_pMIQeSohfoCgZOpC4b9JeYuc7y9waQL6u5rWhwhGvxXeAiYKrKL_tprBQsWFN59GIphsnZ6IhZcYbKnPew_MdziX--C9EKCNhRkpqYiag2&xkcb=SoCA67M3yiFJojSNhR0LbzkdCdPP&fccid=79927877134dd625&vjs=3


In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Load credentials from .env
load_dotenv()

# Database connection info
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# SQL statements
CREATE_TABLE_QUERY = """
CREATE TABLE IF NOT EXISTS indeed_jobs (
    id SERIAL PRIMARY KEY,
    url TEXT,
    job_title TEXT,
    job_location TEXT
);
"""

INSERT_QUERY = """
INSERT INTO indeed_jobs (
    id, url, job_title, job_location
)
VALUES (
    :id, :url, :jobTitle, :jobLocation
) 
RETURNING id;
"""



In [ ]:

# Execute DB operations
try:
    with engine.begin() as conn:  # automatically commits or rolls back
        conn.execute(text(CREATE_TABLE_QUERY))
        job = {}
        job["jobTitle"] = job_title
        job["jobLocation"] = company_location_text
        job["url"] = full_link
            
        conn.execute(text(INSERT_QUERY), job)

        print(f"{job_title} inserted successfully.")

except SQLAlchemyError as e:
    print("Database error:", e)